In [1]:
import os
import pickle
import glob
import random
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoTokenizer, AutoModel, AutoFeatureExtractor
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from tqdm import tqdm

In [2]:
train_df = pd.read_csv('train_data.csv')

In [3]:
train_df

,Unnamed: 0,file_name,examination,indication,technique,comparison,findings,impression,clean_filenames,Files,P-Folder,p-Folder,path,image_path,split
0,0,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,478e39da-ea1ef26f-de4c7167-eb8a0af0-7de6fd2a,files/p10/p10000032/s50414267/02aa804e-bde0afd...,train
1,1,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,71924c95-a7042c53-10843bb7-3febb74d-b597c5f2,files/p10/p10000032/s50414267/174413ec-4ec4c1f...,train
2,2,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,8fb87406-7667e3fe-eced1e60-2adbd40a-63558751,files/p10/p10000032/s53189527/2a2277a9-b0ded15...,train
3,3,s51155740.txt,NaN,History: ___M with SOB during rifaximin infusi...,Chest PA and lateral,None.,Frontal and lateral chest radiographs demonst...,Elevation of the left hemidiaphragm with atel...,s51155740,files,p19,p19007931,bcf8fa5a-f17ea507-cff20d4d-6eeb3c66-e52796d8,files/p10/p10000032/s53189527/e084de3b-be89b11...,train
4,4,s58348111.txt,NaN,Newly diagnosed rheumatoid arthritis. Evaluat...,NaN,No comparison available at the time of dictation.,pleural effusions. Normal size of the cardia...,NaN,s58348111,files,p19,p19561401,fb0732a9-c3d578e5-30f56374-71357f8e-84c9c972,files/p10/p10000032/s53911762/68b5c4b1-227d048...,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368955,377105,s52688542.txt,CHEST (PORTABLE AP)CHEST (PORTABLE AP)i,___ year old woman with ETT // eval infiltrate,NaN,"Chest radiograph since ___, most recently ___",NaN,Mild pulmonary edema is new. Severe consolid...,s52688542,files,p12,p12057859,9e49c4e2-518c7a0a-a7e30554-3b7f5f03-4f71e1e4,files/p19/p19999733/s57132437/428e2c18-5721d8f...,train
368956,377106,s59179690.txt,CHEST,___F with AMS // pneumonia,NaN,___,AP upright and lateral views of the chest pro...,Cardiomegaly unchanged with mild pulmonary co...,s59179690,files,p12,p12057859,b71026b6-807374bf-47f0f0d7-4e984ae4-7f6aaaf3,files/p19/p19999733/s57132437/58c403aa-35ff8bd...,train
368957,377107,s59179690.txt,CHEST,___F with AMS // pneumonia,NaN,___,AP upright and lateral views of the chest pro...,Cardiomegaly unchanged with mild pulmonary co...,s59179690,files,p12,p12057859,f677232c-4f48f565-3193bbb1-b1ee6f4b-37e7c6af,files/p19/p19999987/s55368167/58766883-376a15c...,train
368958,377108,s58463101.txt,CHEST (PORTABLE AP)CHEST (PORTABLE AP),"___F p/w left hemiparesis, found to have b/l S...",NaN,Chest radiographs ___,NaN,Pulmonary and mediastinal vascular congestion...,s58463101,files,p12,p12057859,ccfa71dc-e4d15466-609334a9-ce03f0aa-af08595c,files/p19/p19999987/s58621812/7ba273af-3d290f8...,train


In [4]:
device = torch.device("cuda")
print(f"Using device: {device}")

Using device: cuda


In [5]:
# Pretrained Vision Transformer model for feature extraction
vision_model_name = "google/vit-base-patch16-224-in21k"
feature_extractor = AutoFeatureExtractor.from_pretrained(vision_model_name)
vision_model = AutoModel.from_pretrained(vision_model_name).to(device)
vision_model.eval()

/home/bvs9764/.local/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUAct

In [6]:
# class MIMICImageDataset(Dataset):
#     """Dataset to load MIMIC-CXR-JPG images recursively from a directory."""
#     def __init__(self, image_dir, transform=None):
#         self.image_files = image_dir
#         self.image_files.sort()  # Ensure consistent order
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_path = self.image_files[idx]
#         image = Image.open(img_path).convert("RGB")
#         if self.transform:
#             image = self.transform(image)
#         return image, os.path.relpath(img_path)

In [7]:
class MIMICImageDataset(Dataset):
    """Dataset to load MIMIC-CXR-JPG images from a list of image paths."""
    def __init__(self, image_files, transform=None):
        # Assuming image_files is a list of image paths
        self.image_files = sorted(image_files)  # Ensure consistent order
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, os.path.relpath(img_path)

In [8]:
def create_transform(feature_extractor):
    size = feature_extractor.size
    height = size["height"]
    width = size["width"]
    print("Height: {}".format(height))
    print("Width: {}".format(width))
    normalize = transforms.Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
    flip = transforms.RandomHorizontalFlip(p=0.5)
    return transforms.Compose([
        transforms.Resize((height, width)),
        transforms.ToTensor(),
        normalize,
    ])

In [9]:
image_transform = create_transform(feature_extractor)

Height: 224
Width: 224


In [10]:
# Update this path to your local dataset directory
image_dir = []
for path in train_df["image_path"]:
    image_dir.append( "/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/"+ path)



In [11]:
image_dir

['/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53189527/2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53189527/e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53911762/68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53911762/fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s56699142/ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c.jpg',
 '/scratch/bvs9764/p

In [12]:
# Create dataset and loader
dataset = MIMICImageDataset(image_dir, transform=image_transform)

In [13]:
image_dir

['/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s50414267/174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53189527/2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53189527/e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53911762/68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s53911762/fffabebf-74fd3a1f-673b6b41-96ec0ac9-2ab69818.jpg',
 '/scratch/bvs9764/physionet.org/files/mimic-cxr-jpg/2.1.0/files/p10/p10000032/s56699142/ea030e7a-2e3b1346-bc518786-7a8fd698-f673b44c.jpg',
 '/scratch/bvs9764/p

In [14]:
# class SubsetImageDataset(Dataset):
#     """A subset dataset that uses a slice of the image_files from the original dataset."""
#     def __init__(self, image_files, transform=None):
#         self.image_files = image_files
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_path = self.image_files[idx]
#         image = Image.open(img_path).convert("RGB")
#         if self.transform:
#             image = self.transform(image)
#         return image, os.path.relpath(img_path)

# def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="train_image_embeddings.pkl"):
#     """Process image data in batches, generating and saving embeddings."""

#     # Check if a partial file already exists
#     if os.path.exists(output_path):
#         with open(output_path, "rb") as f:
#             processed_data = pickle.load(f)
#             print(f"Loaded {len(processed_data)} already processed entries.")
#     else:
#         processed_data = {}

#     # Determine how many entries have been processed so far
#     start_index = len(processed_data)
#     # Get the unprocessed image paths
#     unprocessed_files = full_dataset.image_files[start_index:]

#     print(f"Starting from index {start_index}, processing {len(unprocessed_files)} entries.")

#     if len(unprocessed_files) == 0:
#         print("No unprocessed images found.")
#         return processed_data

#     # Create a subset dataset for the unprocessed portion
#     subset_dataset = SubsetImageDataset(unprocessed_files, transform=image_transform)
#     loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#     for batch_i, (images, filenames) in enumerate(tqdm(loader, desc="Processing batches")):
#         images = images.to(device)

#         # Generate embeddings for the current batch
#         with torch.no_grad():
#             outputs = model(pixel_values=images)
#             batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

#         # Calculate the batch indices
#         batch_start_idx = start_index + batch_i * batch_size
#         batch_end_idx = batch_start_idx + len(filenames)
#         batch_indices = range(batch_start_idx, batch_end_idx)

#         # Update processed_data dictionary
#         for idx, fname, emb in zip(batch_indices, filenames, batch_embeddings):
#             processed_data[idx] = {
#                 "filename": fname,
#                 "embedding": emb
#             }

#         # Save progress to file after each batch
#         with open(output_path, "wb") as f:
#             pickle.dump(processed_data, f)

#         print(f"Saved up to index {batch_end_idx - 1}.")

#     print("All embeddings processed and saved.")
#     return processed_data

In [15]:
# image_embeddings = process_embeddings_in_batches(dataset, vision_model, batch_size=64)

In [16]:
# class SubsetImageDataset(Dataset):
#     """A subset dataset that uses a slice of the image_files from the original dataset."""
#     def __init__(self, image_files, transform=None):
#         self.image_files = image_files
#         self.transform = transform

#     def __len__(self):
#         return len(self.image_files)

#     def __getitem__(self, idx):
#         img_path = self.image_files[idx]
#         if not os.path.exists(img_path):
#             raise FileNotFoundError(img_path)
#         image = Image.open(img_path).convert("RGB")
#         if self.transform:
#             image = self.transform(image)
#         return image, os.path.relpath(img_path)

# def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="train_image_embeddings.pkl"):
#     """Process image data in batches, skipping missing files and recording them for later processing."""

#     # Check if a partial file already exists
#     if os.path.exists(output_path):
#         with open(output_path, "rb") as f:
#             processed_data = pickle.load(f)
#             print(f"Loaded {len(processed_data)} already processed entries.")
#     else:
#         processed_data = {}

#     # Determine how many entries have been processed so far
#     start_index = len(processed_data)
#     # Get the unprocessed image paths
#     unprocessed_files = full_dataset.image_files[start_index:]

#     print(f"Starting from index {start_index}, processing {len(unprocessed_files)} entries.")

#     if len(unprocessed_files) == 0:
#         print("No unprocessed images found.")
#         return processed_data

#     # Create a subset dataset for the unprocessed portion
#     subset_dataset = SubsetImageDataset(unprocessed_files, transform=image_transform)
#     loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

#     # Keep track of missing files
#     missing_files = []

#     for batch_i, (images, filenames) in enumerate(tqdm(loader, desc="Processing batches")):
#         try:
#             images = images.to(device)

#             # Generate embeddings for the current batch
#             with torch.no_grad():
#                 outputs = model(pixel_values=images)
#                 batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

#             # Calculate the batch indices
#             batch_start_idx = start_index + batch_i * batch_size
#             batch_end_idx = batch_start_idx + len(filenames)
#             batch_indices = range(batch_start_idx, batch_end_idx)

#             # Update processed_data dictionary
#             for idx, fname, emb in zip(batch_indices, filenames, batch_embeddings):
#                 processed_data[idx] = {
#                     "filename": fname,
#                     "embedding": emb
#                 }

#             # Save progress to file after each batch
#             with open(output_path, "wb") as f:
#                 pickle.dump(processed_data, f)

#             print(f"Saved up to index {batch_end_idx - 1}.")
#         except FileNotFoundError as e:
#             missing_files.append(str(e))
#             print(f"File missing: {e}")

#     # Log missing files for later processing
#     if missing_files:
#         missing_log_path = "missing_files.log"
#         with open(missing_log_path, "w") as log_file:
#             log_file.write("\n".join(missing_files))
#         print(f"Logged {len(missing_files)} missing files to '{missing_log_path}'.")

#     print("All embeddings processed and saved.")
#     return processed_data, missing_files

In [17]:
class SubsetImageDataset(Dataset):
    """A subset dataset that uses a slice of the image_files from the original dataset."""
    def __init__(self, image_files, transform=None):
        self.image_files = image_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        try:
            if not os.path.exists(img_path):
                raise FileNotFoundError(img_path)
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, os.path.relpath(img_path)
        except FileNotFoundError as e:
            # Skip missing files
            return None, None


In [18]:
def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="train_image_embeddings.pkl"):
    """Process image data in batches, skipping missing files and recording them for later processing."""

    if os.path.exists(output_path):
        with open(output_path, "rb") as f:
            processed_data = pickle.load(f)
            print(f"Loaded {len(processed_data)} already processed entries.")
    else:
        processed_data = {}

    start_index = len(processed_data)
    unprocessed_files = full_dataset.image_files[start_index:]

    print(f"Starting from index {start_index}, processing {len(unprocessed_files)} entries.")

    if len(unprocessed_files) == 0:
        print("No unprocessed images found.")
        return processed_data

    subset_dataset = SubsetImageDataset(unprocessed_files, transform=image_transform)
    loader = DataLoader(subset_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    missing_files = []

    for batch_i, (images, filenames) in enumerate(tqdm(loader, desc="Processing batches")):
        # Filter out missing files
        valid_indices = [i for i, (img, fname) in enumerate(zip(images, filenames)) if img is not None and fname is not None]
        if not valid_indices:
            continue

        images = torch.stack([images[i] for i in valid_indices]).to(device)
        filenames = [filenames[i] for i in valid_indices]

        try:
            with torch.no_grad():
                outputs = model(pixel_values=images)
                batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

            batch_start_idx = start_index + batch_i * batch_size
            batch_end_idx = batch_start_idx + len(filenames)
            batch_indices = range(batch_start_idx, batch_end_idx)

            for idx, fname, emb in zip(batch_indices, filenames, batch_embeddings):
                processed_data[idx] = {
                    "filename": fname,
                    "embedding": emb
                }

            with open(output_path, "wb") as f:
                pickle.dump(processed_data, f)

            print(f"Saved up to index {batch_end_idx - 1}.")
        except Exception as e:
            print(f"Error processing batch {batch_i}: {e}")

    # Log missing files
    for img_path in subset_dataset.image_files:
        if not os.path.exists(img_path):
            missing_files.append(img_path)

    if missing_files:
        missing_log_path = "missing_files.log"
        with open(missing_log_path, "w") as log_file:
            log_file.write("\n".join(missing_files))
        print(f"Logged {len(missing_files)} missing files to '{missing_log_path}'.")

    print("All embeddings processed and saved.")
    return processed_data, missing_files


In [19]:
# # THE ERROR

# import os
# from torch.utils.data import DataLoader, Dataset
# from PIL import Image
# from torchvision import transforms
# from tqdm import tqdm

# # Custom Dataset
# class CustomDataset(Dataset):
#     def __init__(self, file_paths, transform=None):
#         self.file_paths = file_paths
#         self.transform = transform

#     def __len__(self):
#         return len(self.file_paths)

#     def __getitem__(self, idx):
#         img_path = self.file_paths[idx]
#         if not os.path.exists(img_path):
#             return None, None  # Return None for missing files
#         try:
#             image = Image.open(img_path).convert("RGB")
#             if self.transform:
#                 image = self.transform(image)
#             return image, img_path
#         except Exception as e:
#             return None, None  # Return None for unreadable files

# # Custom collate function to skip None entries
# def custom_collate_fn(batch):
#     filtered_batch = [item for item in batch if item[0] is not None]
#     if not filtered_batch:
#         return torch.empty(0), []  # Return empty batch if all entries are None
#     images, filenames = zip(*filtered_batch)
#     return torch.stack(images), list(filenames)

# def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="embeddings.pt"):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()

#     loader = DataLoader(
#         full_dataset, 
#         batch_size=batch_size, 
#         shuffle=False, 
#         num_workers=4, 
#         pin_memory=True, 
#         collate_fn=custom_collate_fn
#     )

#     embeddings = []
#     missing_files = []

#     for batch_i, (images, filenames) in enumerate(tqdm(loader, desc="Processing batches")):
#         if images.nelement() == 0:  # Skip empty batches
#             missing_files.extend(filenames)  # Add skipped files to missing list
#             continue

#         images = images.to(device)
#         with torch.no_grad():
#             output = model(images)  # Forward pass
#             if hasattr(output, 'pooler_output'):
#                 batch_embeddings = output.pooler_output  # Use pooled embeddings
#             elif hasattr(output, 'last_hidden_state'):
#                 batch_embeddings = output.last_hidden_state[:, 0, :]  # Use CLS token embeddings
#             else:
#                 raise ValueError("Unsupported model output structure.")
#         embeddings.append(batch_embeddings.cpu())

#     # Save the embeddings
#     torch.save(torch.cat(embeddings), output_path)
#     print(f"Embeddings saved to {output_path}")
#     print(f"Missing files: {len(missing_files)}")
#     return torch.cat(embeddings), missing_files



In [20]:
# import os
# import pickle
# import torch
# from torch.utils.data import DataLoader, Dataset
# from PIL import Image
# from torchvision import transforms
# from tqdm import tqdm

# # Custom Dataset
# class CustomDataset(Dataset):
#     def __init__(self, file_paths, transform=None):
#         self.file_paths = file_paths
#         self.transform = transform

#     def __len__(self):
#         return len(self.file_paths)

#     def __getitem__(self, idx):
#         img_path = self.file_paths[idx]
#         if not os.path.exists(img_path):
#             return None, None  # Return None for missing files
#         try:
#             image = Image.open(img_path).convert("RGB")
#             if self.transform:
#                 image = self.transform(image)
#             return image, img_path
#         except Exception as e:
#             return None, None  # Return None for unreadable files

# # Custom collate function to skip None entries
# def custom_collate_fn(batch):
#     filtered_batch = [item for item in batch if item[0] is not None]
#     if not filtered_batch:
#         return torch.empty(0), []  # Return empty batch if all entries are None
#     images, filenames = zip(*filtered_batch)
#     return torch.stack(images), list(filenames)

# def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="train_image_embeddings.pkl"):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()

#     loader = DataLoader(
#         full_dataset, 
#         batch_size=batch_size, 
#         shuffle=False, 
#         num_workers=4, 
#         pin_memory=True, 
#         collate_fn=custom_collate_fn
#     )

#     missing_files = []

#     for batch_i, (images, filenames) in enumerate(tqdm(loader, desc="Processing batches")):
#         if images.nelement() == 0:  # Skip empty batches
#             missing_files.extend(filenames)  # Add skipped files to missing list
#             continue

#         images = images.to(device)
#         with torch.no_grad():
#             output = model(images)  # Forward pass
#             if hasattr(output, 'pooler_output'):
#                 batch_embeddings = output.pooler_output.cpu()  # Use pooled embeddings
#             elif hasattr(output, 'last_hidden_state'):
#                 batch_embeddings = output.last_hidden_state[:, 0, :].cpu()  # Use CLS token embeddings
#             else:
#                 raise ValueError("Unsupported model output structure.")

#         # Save each batch incrementally
#         with open(output_path, 'ab') as f:
#             pickle.dump((batch_embeddings, filenames), f)

#     print(f"Embeddings incrementally saved to {output_path}")
#     print(f"Missing files: {len(missing_files)}")
#     return output_path, missing_files


In [21]:
# image_embeddings = process_embeddings_in_batches(dataset, vision_model, batch_size=16)

In [22]:
# import os
# import pickle
# import torch
# from torch.utils.data import DataLoader, Dataset
# from PIL import Image
# from torchvision import transforms
# from tqdm import tqdm

# # Custom Dataset
# class CustomDataset(Dataset):
#     def __init__(self, file_paths, transform=None):
#         self.file_paths = file_paths
#         self.transform = transform

#     def __len__(self):
#         return len(self.file_paths)

#     def __getitem__(self, idx):
#         img_path = self.file_paths[idx]
#         if not os.path.exists(img_path):
#             return None, img_path  # Return None for image, but return path to log missing files
#         try:
#             image = Image.open(img_path).convert("RGB")
#             if self.transform:
#                 image = self.transform(image)
#             return image, img_path
#         except Exception as e:
#             return None, img_path  # Return None for unreadable files

# # Custom collate function to skip None entries
# def custom_collate_fn(batch):
#     filtered_batch = [item for item in batch if item[0] is not None]
#     missing_files = [item[1] for item in batch if item[0] is None]  # Collect missing file paths
#     if not filtered_batch:
#         return torch.empty(0), missing_files  # Return empty batch if all entries are None
#     images, filenames = zip(*filtered_batch)
#     return torch.stack(images), filenames, missing_files

# def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="train_image_embeddings.pkl"):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)
#     model.eval()

#     loader = DataLoader(
#         full_dataset, 
#         batch_size=batch_size, 
#         shuffle=False, 
#         num_workers=4, 
#         pin_memory=True, 
#         collate_fn=custom_collate_fn
#     )

#     all_missing_files = []  # List to accumulate missing files

#     for batch_i, (images, filenames, missing_files) in enumerate(tqdm(loader, desc="Processing batches")):
#         all_missing_files.extend(missing_files)  # Track all missing files across batches

#         if images.nelement() == 0:  # Skip empty batches
#             continue

#         images = images.to(device)
#         with torch.no_grad():
#             output = model(images)  # Forward pass
#             if hasattr(output, 'pooler_output'):
#                 batch_embeddings = output.pooler_output.cpu()  # Use pooled embeddings
#             elif hasattr(output, 'last_hidden_state'):
#                 batch_embeddings = output.last_hidden_state[:, 0, :].cpu()  # Use CLS token embeddings
#             else:
#                 raise ValueError("Unsupported model output structure.")

#         # Save each batch incrementally
#         with open(output_path, 'ab') as f:
#             pickle.dump((batch_embeddings, filenames), f)

#     print(f"Embeddings incrementally saved to {output_path}")
#     print(f"Total missing files: {len(all_missing_files)}")
#     return output_path, all_missing_files



In [23]:
# image_embeddings = process_embeddings_in_batches(dataset, vision_model, batch_size=16)


In [24]:
import os
import pickle
import torch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torchvision import transforms
from tqdm import tqdm

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, file_paths, transform=None):
        # Filter out non-existent files during initialization
        self.file_paths = [fp for fp in file_paths if os.path.exists(fp)]
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img_path = self.file_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, img_path
        except Exception as e:
            # Log the exception for debugging
            print(f"Error reading file {img_path}: {e}")
            return None, img_path

# Custom collate function to skip None entries
def custom_collate_fn(batch):
    filtered_batch = [item for item in batch if item[0] is not None]
    missing_files = [item[1] for item in batch if item[0] is None]  # Collect missing file paths
    if not filtered_batch:
        return torch.empty(0), [], missing_files  # Return empty batch if all entries are None
    images, filenames = zip(*filtered_batch)
    return torch.stack(images), filenames, missing_files

def process_embeddings_in_batches(full_dataset, model, batch_size=64, output_path="train_image_embeddings.pkl"):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    loader = DataLoader(
        full_dataset, 
        batch_size=batch_size, 
        shuffle=False, 
        num_workers=2,  # Reduce workers as suggested by the warning
        pin_memory=True, 
        collate_fn=custom_collate_fn
    )

    all_missing_files = []  # List to accumulate missing files

    for batch_i, (images, filenames, missing_files) in enumerate(tqdm(loader, desc="Processing batches")):
        all_missing_files.extend(missing_files)  # Track all missing files across batches

        if images.nelement() == 0:  # Skip empty batches
            continue

        images = images.to(device)
        with torch.no_grad():
            output = model(images)  # Forward pass
            if hasattr(output, 'pooler_output'):
                batch_embeddings = output.pooler_output.cpu()  # Use pooled embeddings
            elif hasattr(output, 'last_hidden_state'):
                batch_embeddings = output.last_hidden_state[:, 0, :].cpu()  # Use CLS token embeddings
            else:
                raise ValueError("Unsupported model output structure.")

        # Save each batch incrementally
        with open(output_path, 'ab') as f:
            pickle.dump((batch_embeddings, filenames), f)

    print(f"Embeddings incrementally saved to {output_path}")
    print(f"Total missing files: {len(all_missing_files)}")
    if all_missing_files:
        print(f"Missing files are logged in `missing_files.log`.")
        with open("missing_files.log", "w") as log_file:
            log_file.write("\n".join(all_missing_files))

    return output_path, all_missing_files

# Assuming `file_paths` is a list of image paths
# and `vision_model` is your pre-trained model
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Initialize dataset and DataLoader
dataset = CustomDataset(file_paths, transform=transform)
output_path, missing_files = process_embeddings_in_batches(dataset, vision_model, batch_size=16)


NameError: name 'file_paths' is not defined

In [25]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = self.labels[idx]
            return image, label, img_path  # Include the path for debugging
        except FileNotFoundError:
            print(f"File not found: {img_path}")  # Log the missing file
            return None  # Skip this item

    def __len__(self):
        return len(self.image_paths)


In [26]:
from torch.utils.data import DataLoader

loader = DataLoader(
    dataset=CustomDataset(image_paths, labels),
    batch_size=16,
    num_workers=2,  # Adjust for your system
    collate_fn=custom_collate_fn  # Use custom collate function
)

for batch in loader:
    if batch is None:
        continue  # Skip empty batches
    images, labels, filenames = batch
    # Process your valid batches


NameError: name 'image_paths' is not defined

In [27]:
# Updated __getitem__ to handle missing files
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels):
        self.image_paths = image_paths
        self.labels = labels

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            label = self.labels[idx]
            return image, label, img_path  # Include img_path for debugging
        except FileNotFoundError:
            return None  # Skip missing file

    def __len__(self):
        return len(self.image_paths)


# Custom collate function to filter out None values
def custom_collate_fn(batch):
    # Filter out None items
    batch = [item for item in batch if item is not None]
    if len(batch) == 0:
        return None  # Skip empty batches
    return default_collate(batch)


# Process embeddings in batches while handling missing files
def process_embeddings_in_batches(full_dataset, model, batch_size=16, output_path="embeddings.npy"):
    loader = DataLoader(
        full_dataset,
        batch_size=batch_size,
        num_workers=2,  # Adjust workers for system limits
        collate_fn=custom_collate_fn,
    )

    all_missing_files = []  # Track missing files

    for batch_i, batch in enumerate(tqdm(loader, desc="Processing batches")):
        if batch is None:
            continue  # Skip empty batches
        images, labels, img_paths = batch
        embeddings = model(images)

        # Save embeddings and filenames as needed
        # Track missing files
        all_missing_files.extend([path for path in img_paths if path is None])

    print(f"Total missing files: {len(all_missing_files)}")



In [28]:
import torch
import numpy as np

# Function to save embeddings to a file (e.g., numpy array)
def save_embeddings(embeddings, filenames, output_path):
    """
    Save the embeddings to a numpy file.
    
    Args:
        embeddings (list): List of embeddings (Tensor).
        filenames (list): List of corresponding image file paths.
        output_path (str): Path where the embeddings will be saved.
    """
    embeddings = torch.stack(embeddings).cpu().numpy()  # Convert embeddings to numpy array
    filenames = np.array(filenames)  # Convert filenames to numpy array
    np.savez(output_path, embeddings=embeddings, filenames=filenames)

# Process embeddings in batches and save them
def process_embeddings_in_batches(full_dataset, model, batch_size=16, output_path="embeddings.npz"):
    loader = DataLoader(
        full_dataset,
        batch_size=batch_size,
        num_workers=2,  # Adjust workers for system limits
        collate_fn=custom_collate_fn,
    )

    all_embeddings = []  # List to store embeddings
    all_filenames = []  # List to store corresponding filenames
    all_missing_files = []  # Track missing files

    for batch_i, batch in enumerate(tqdm(loader, desc="Processing batches")):
        if batch is None:
            continue  # Skip empty batches

        images, labels, img_paths = batch

        # Generate embeddings using the model
        with torch.no_grad():  # Disable gradient computation
            embeddings = model(images)

        # Append embeddings and filenames to the lists
        all_embeddings.append(embeddings)
        all_filenames.extend(img_paths)  # Add the filenames associated with the embeddings

        # Track missing files
        all_missing_files.extend([path for path in img_paths if path is None])

    # Concatenate all embeddings from different batches
    all_embeddings = torch.cat(all_embeddings, dim=0)

    # Save embeddings and filenames to a file
    save_embeddings(all_embeddings, all_filenames, output_path)

    print(f"Total missing files: {len(all_missing_files)}")
    print(f"Embeddings saved to: {output_path}")


In [29]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Function to save embeddings to a file (e.g., numpy array)
def save_embeddings(embeddings, filenames, output_path):
    """
    Save the embeddings to a numpy file.
    
    Args:
        embeddings (list): List of embeddings (Tensor).
        filenames (list): List of corresponding image file paths.
        output_path (str): Path where the embeddings will be saved.
    """
    embeddings = torch.stack(embeddings).cpu().numpy()  # Convert embeddings to numpy array
    filenames = np.array(filenames)  # Convert filenames to numpy array
    np.savez(output_path, embeddings=embeddings, filenames=filenames)

# Custom collate function for batching (to handle any padding or special cases)
def custom_collate_fn(batch):
    images, labels, img_paths = zip(*batch)
    images = torch.stack(images)  # Stack images into a tensor
    return images, labels, img_paths

# Function to process embeddings in batches and save them
def process_embeddings_in_batches(full_dataset, model, batch_size=16, output_path="embeddings.npz"):
    loader = DataLoader(
        full_dataset,
        batch_size=batch_size,
        num_workers=2,  # Adjust workers for system limits
        collate_fn=custom_collate_fn,
    )

    all_embeddings = []  # List to store embeddings
    all_filenames = []  # List to store corresponding filenames
    all_missing_files = []  # Track missing files

    for batch_i, batch in enumerate(tqdm(loader, desc="Processing batches")):
        if batch is None:
            continue  # Skip empty batches

        images, labels, img_paths = batch

        # Generate embeddings using the model
        with torch.no_grad():  # Disable gradient computation
            embeddings = model(images)

        # Append embeddings and filenames to the lists
        all_embeddings.append(embeddings)
        all_filenames.extend(img_paths)  # Add the filenames associated with the embeddings

        # Track missing files
        all_missing_files.extend([path for path in img_paths if path is None])

    # Concatenate all embeddings from different batches
    all_embeddings = torch.cat(all_embeddings, dim=0)

    # Save embeddings and filenames to a file
    save_embeddings(all_embeddings, all_filenames, output_path)

    print(f"Total missing files: {len(all_missing_files)}")
    print(f"Embeddings saved to: {output_path}")


# Example invocation to start the process:

# Assuming `full_dataset` is your dataset and `vision_model` is the model you've defined
output_path = "/path/to/output/embeddings.npz"
process_embeddings_in_batches(full_dataset, vision_model, batch_size=16, output_path=output_path)


NameError: name 'full_dataset' is not defined